# Update Commissioners 2022

After redistricting, all commissioners are in new districts (all districts have new IDs), so they all need a new row in the commissioners table. 

A medium-term project will be upgrading the commissioners table so there is one row per commissioner-term.

In [1]:
import os
os.chdir('../')

import pytz
import pandas as pd
from datetime import datetime

In [2]:
election_year = 2024

# The dates go into the google sheet as text, they don't need to be datetime
start_date = '2025-01-02'
end_date = '2027-01-02'

In [3]:
people = pd.read_csv('data/people.csv')
results = pd.read_csv('data/dcboe/candidate_votes.csv')
write_in_winners = pd.read_csv('data/dcboe/write_in_winners.csv')
lookup = pd.read_csv('data/external_id_lookup.csv')

In [4]:
results = results[results.election_year == election_year].copy()
write_in_winners = write_in_winners[write_in_winners.election_year == election_year].copy()

In [5]:
winners = pd.concat([
    results[(results.winner) & (~results.write_in_winner)][['external_id', 'smd_id']]
    , write_in_winners[['external_id', 'smd_id']]
]).sort_values(by='smd_id')

winners

,external_id,smd_id
1418,dfb199b30d1d440188a7cfe1aff8b6fef3fc7d5549b662...,smd_2022_1A01
1420,a4c839b09ed0da89280a5e0ffb7ceaf3ce8f9138ee3dc2...,smd_2022_1A02
1422,343ea37f82449589debc04c14dbb229f69873f3f219ec7...,smd_2022_1A03
1424,53c501c6627907f657d2faae46273d0c32597fab996ff6...,smd_2022_1A04
1426,4d16b00a6a09af3d0efaef769aa6ac664c7e482fd5525d...,smd_2022_1A05
...,...,...
2095,8b116b303533f51ac0e97ef9e93112c23c865b2c835d95...,smd_2022_8F01
2097,9053a4ef792c97c909ce5d75ca71df598bd18543a834d3...,smd_2022_8F02
2101,87edc46acf64161d76a169d8a8e0b294e1fb93c58709a1...,smd_2022_8F03
2103,96c0e6fbb6a60be385eda74d59677765d2ab6bd32aed04...,smd_2022_8F04


In [6]:
winners_lookup = pd.merge(winners, lookup, how='inner', on='external_id').drop('full_name', axis=1)
winners_lookup_people = pd.merge(winners_lookup, people, how='inner', on='person_id')

In [7]:
# Confirm that there is only one winner per SMD
winners_lookup_people.groupby('smd_id').size().max() == 1

True

In [8]:
winners_lookup_people['start_date'] = start_date
winners_lookup_people['end_date'] = end_date

In [9]:
columns_to_clipboard = ['person_id', 'smd_id', 'full_name', 'start_date', 'end_date']
winners_lookup_people[columns_to_clipboard]

,person_id,smd_id,full_name,start_date,end_date
0,11707,smd_2022_1A01,Jaspal Bhatia,2025-01-02,2027-01-02
1,10285,smd_2022_1A02,Dieter Lehmann Morales,2025-01-02,2027-01-02
2,11345,smd_2022_1A03,Carlo Perri,2025-01-02,2027-01-02
3,11371,smd_2022_1A04,Jeremy Sherman,2025-01-02,2027-01-02
4,10005,smd_2022_1A05,Christine Miller,2025-01-02,2027-01-02
...,...,...,...,...,...
312,11542,smd_2022_8F01,Nic Wilson,2025-01-02,2027-01-02
313,11849,smd_2022_8F02,Markita Bryant,2025-01-02,2027-01-02
314,11406,smd_2022_8F03,Brian Strege,2025-01-02,2027-01-02
315,10210,smd_2022_8F04,Edward Daniels,2025-01-02,2027-01-02


In [10]:
# Copy to "commissioners" tab
winners_lookup_people[columns_to_clipboard].to_clipboard(index=False)